# USING OPENCV VIRTUAL PEN  

In [ ]:
#installing libraries

import cv2
import numpy as np
import time
# if you want to load hsv from disk use this
load_from_disk = True
if load_from_disk:
    hsv_value = np.load('hsv_value.npy')
#capturing video from webcam
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

kernel_Value = np.ones((5,5),np.uint8)

# Creating a Canvas1
Canvas1 = None

# creating points
x1,y1=0,0

# setting up threshhold for noise
noiseth = 800

while(1):
    _, frame = cap.read()
    frame = cv2.flip( frame, 1 )
    
    # Initialize the Canvas1 as a black image of the same size as the frame.
    if Canvas1 is None:
        Canvas1 = np.zeros_like(frame)

    # Converting BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # If you're reading from memory then load the upper and lower ranges of pen 
    # from there
    if load_from_disk:
        lower = hsv_value[0]
        upper = hsv_value[1]
            
    # Otherwise define your own custom values for upper and lower range of the pen.
    else:           
        lower  = np.array([134, 20, 204])
        upper = np.array([179, 255, 255])
    
    mask = cv2.inRange(hsv, lower, upper)
    
    # Perform morphological operations to get rid of the noise(noise reduction)
    mask = cv2.erode(mask,kernel_Value,iterations = 1)
    mask = cv2.dilate(mask,kernel_Value,iterations = 2)
    
    # Finding contours
    cnt, hierarchy = cv2.findcnt(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Make sure there is a contour present and also its size is bigger than 
    # noise threshold.
    if cnt and cv2.contourArea(max(cnt, 
                                 key = cv2.contourArea)) > noiseth:
                
        c = max(cnt, key = cv2.contourArea)    
        x2,y2,w,h = cv2.boundingRect(c)
        
        # If there were no previous points then save the detected x2,y2 
        # coordinates as x1,y1. 
        # This is true when we writing for the first time or when writing 
        # again when the pen had disappeared from view.
        if x1 == 0 and y1 == 0:
            x1,y1= x2,y2
            
        else:
            # Drawing the line on Canvas1
            Canvas1 = cv2.line(Canvas1, (x1,y1),(x2,y2), [255,0,0], 4)
        
        # After the line is drawn the new points become the previous points.
        x1,y1= x2,y2

    else:
        #setting x1,y1 = 0, If there were no cnt detected  
        x1,y1 =0,0
    
    # Merging the Canvas1 and the frame.
    frame = cv2.add(frame,Canvas1)
    
    # Optionally (stack both frames.)
    stacked = np.hstack((Canvas1,frame))
    cv2.imshow('VIRTUAL PEN',cv2.resize(stacked,None,fx=0.6,fy=0.6))

    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
    # When c is pressed clear the Canvas1
    if k == ord('c'):
        Canvas1 = None

cv2.destroyAllWindows()
cap.release()